In [1]:
import torch
import numpy as np
import os
import matplotlib as plt
from copy import deepcopy
from experiment_with_t import *

In [2]:
samples = []
bdry_samples = []
n = 1000
for i in range(n):
    samples.append(sample(K))
    bdry_samples.append(sample(K, t=1.))


best_loss = float('inf')
for model_path in os.listdir('models'):
    PATH = f'models/{model_path}'
    model = model_path.split(',')
    if model[-1][-3:] != '.pt':
        continue
    else:
        hl_size, reg, lr, batch_size, iters = model
        hl_size = int(hl_size)
        reg = float(reg)
        lr = float(lr)
        batch_size = int(batch_size)
        iters = int(iters[:-3])

        net = BigNet(K, hl_size)
        net.load_state_dict(torch.load(PATH))

        loss = 0.
        bdry_loss = 0.
        for state, bdry_state in zip(samples, bdry_samples):
            t, s, q = state
            _, bdry_s, bdry_q = bdry_state
            loss += hjb_term(net, t, s, q, lam, p, K)
            bdry_loss += hjb_bdry(net, s, q, p)
        
        total_loss = loss + bdry_loss
        if total_loss < best_loss:
            best_dict = {
                'net':net,
                'hl_size':hl_size,
                'reg':reg,
                'lr':lr,
                'batch_size':batch_size,
                'iters':iters,
                'loss':loss/n,
                'bdry_loss':bdry_loss/n,
                'total_loss':total_loss/n
            }

In [3]:
best_dict

{'net': BigNet(
   (fc1): Linear(in_features=9, out_features=30, bias=True)
   (fc2): Linear(in_features=30, out_features=30, bias=True)
   (fc3): Linear(in_features=30, out_features=1, bias=True)
 ),
 'hl_size': 30,
 'reg': 0.5,
 'lr': 0.1,
 'batch_size': 1,
 'iters': 10,
 'loss': tensor(1.2590, grad_fn=<DivBackward0>),
 'bdry_loss': tensor([0.0023], grad_fn=<DivBackward0>),
 'total_loss': tensor([1.2613], grad_fn=<DivBackward0>)}

In [5]:
from experiment_with_t import policy
t, s, q = sample(K)
policy(net, t, s, q, K)

tensor([0.1848, 0.2888, 0.2753, 0.2511], grad_fn=<SoftmaxBackward>)